## Save / Restore : 최종 저장 시점인 epoch 값(global_step)도 저장/복원해보자

참고 : http://stackoverflow.com/questions/36113090/tensorflow-get-the-global-step-when-restoring-checkpoints

보완할 부분 : 저장 파일명(XXX : global_step) model-XXX.cptk.data-00000-of-00001에서 data-00000-of-00001 부분[shard 정보] 탐구)

또다른 구현 사례(별도 변수로 처리) : https://github.com/hunkim/DeepLearningZeroToAll/blob/master/lab-13-3-mnist_save_restore.py

In [1]:
import tensorflow as tf
import os

In [2]:
LOG_DIR = './SaveAndRestore_log'    # modulename_log 형태로
LOG_PERIOD = 1000    # LOG를 남기는 반복 시점 (대개 epoch와 연계)

In [3]:
epoch_start = 0
epoch_end = 10000

# save/restore시 처리하기 위한 variable 선언  : name='global_step'
global_step = tf.Variable(epoch_start, name='global_step', trainable=False)

### sample로 linear_regression 수행 (optimizer.minimize()의 global_step 추가)
참조 : https://github.com/hunkim/DeepLearningZeroToAll/blob/master/lab-02-1-linear_regression.py

In [4]:
# X and Y data
x_train = [1, 2, 3]
y_train = [1, 2, 3]

# Try to find values for W and b to compute y_data = x_data * W + b
# We know that W should be 1 and b should be 0
# But let TensorFlow figure it out
W = tf.Variable(tf.random_normal([1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

# Our hypothesis XW+b
hypothesis = x_train * W + b

# cost/loss function
cost = tf.reduce_mean(tf.square(hypothesis - y_train))

# Minimize
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train = optimizer.minimize(cost, global_step=global_step)

In [5]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    # Saver and Restore
    saver = tf.train.Saver()
    checkpoint = tf.train.get_checkpoint_state(LOG_DIR)
    if checkpoint and checkpoint.model_checkpoint_path:
        try:
            saver.restore(sess, checkpoint.model_checkpoint_path)
            print("Successfully loaded:", checkpoint.model_checkpoint_path)
            # resoter 시 epoch의 시작 위치를 restore한 값으로 초기화 (방법1/방법2 중 원하는 것으로 사용)
            # 방법1
#            epoch_start = sess.run(global_step)
            # 방법2
            epoch_start = sess.run(tf.contrib.framework.get_global_step())
            print("epoch_start: ", epoch_start)
        except:
            print("Error on loading old network weights")
    else:
        print("Could not find old network weights")

    for i in range(epoch_start, epoch_end+1):
        sess.run(train)

        # 열심히 학습하자
        if i % 100 == 0:
            print("epoch: ", i)

        if i > 0 and i % LOG_PERIOD == 0:
            print("Saving network...")
            if not os.path.exists(LOG_DIR):
                os.makedirs(LOG_DIR)
            # 저장시 global_step 값을 현재의 epoch 값으로 설정
            saver.save(sess, os.path.join(LOG_DIR, "model.ckpt"), global_step=i)
            print("Saved")

Successfully loaded: ./SaveAndRestore_log\model.ckpt-4000
epoch_start:  4001
epoch:  4100
epoch:  4200
epoch:  4300
epoch:  4400
epoch:  4500
epoch:  4600
epoch:  4700
epoch:  4800
epoch:  4900
epoch:  5000
Saving network...
Saved
epoch:  5100
epoch:  5200
epoch:  5300
epoch:  5400
epoch:  5500
epoch:  5600
epoch:  5700
epoch:  5800
epoch:  5900
epoch:  6000
Saving network...
Saved
epoch:  6100
epoch:  6200
epoch:  6300
epoch:  6400
epoch:  6500
epoch:  6600
epoch:  6700
epoch:  6800
epoch:  6900
epoch:  7000
Saving network...
Saved
epoch:  7100
epoch:  7200
epoch:  7300
epoch:  7400
epoch:  7500
epoch:  7600
epoch:  7700
epoch:  7800
epoch:  7900
epoch:  8000
Saving network...
Saved
epoch:  8100
epoch:  8200
epoch:  8300
epoch:  8400
epoch:  8500
epoch:  8600
epoch:  8700
epoch:  8800
epoch:  8900
epoch:  9000
Saving network...
Saved
epoch:  9100
epoch:  9200
epoch:  9300
epoch:  9400
epoch:  9500
epoch:  9600
epoch:  9700
epoch:  9800
epoch:  9900
epoch:  10000
Saving network...
Save